In [12]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import pickle
from matplotlib import pyplot as plt
import statsmodels.api as sm
from matplotlib.ticker import Formatter
import collections
from decimal import Decimal, getcontext, ROUND_HALF_UP

logSH = pd.read_csv('/mnt/data/20201210/logs_20201210_zt_88_03_day_88data/mdLog_SH_20201210_0832.csv',
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                               "exchange", "time", "cum_volume", "cum_amount", "close",
                                               "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                               "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                               "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                               "ask4q", "ask5q", "openPrice", "numTrades"]]

logSH1 = pd.read_csv('/mnt/data/20201210/zs_7201/zs_7201_20201210_10.0.98.140_postvde/Logs_full/mdLog_SH_20201210_0854.csv',
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                               "exchange", "time", "cum_volume", "cum_amount", "close",
                                               "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                               "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                               "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                               "ask4q", "ask5q", "openPrice", "numTrades"]]
# # 1. data format
print("original data source type: ")
print(logSH["source"].unique())
print("new data source type: ")
print(logSH1["source"].unique())
print(logSH["time"].unique())
print(logSH1["time"].unique())

original data source type: 
[22 23]
new data source type: 
[22 23]
['08:32:59.400' '08:33:00.720' '08:33:02.960' ... '15:10:42.000'
 '15:10:43.000' '15:10:42.480']
['08:54:19.150' '08:54:19.000' '08:54:19.680' ... '15:17:10.000'
 '15:17:11.240' '15:17:11.790']


In [13]:
logSH["time"] = logSH["time"].apply(lambda x: int((x.replace(':', "")).replace(".", "")))
logSH1["time"] = logSH1["time"].apply(lambda x: int((x.replace(':', "")).replace(".", "")))

display(logSH[(logSH["StockID"] == 600004) & (logSH["time"] >= 143000000) & (logSH["time"] <= 145000000)])
display(logSH1[(logSH1["StockID"] == 600004) & (logSH1["time"] >= 143000000) & (logSH1["time"] <= 145000000)])

,clockAtArrival,sequenceNo,source,StockID,exchange,time,cum_volume,cum_amount,close,bid1p,...,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice,numTrades
11857797,1607581806032280,104105575,23,600004,SH,143001000,5438227,78301010.81,14.39,14.37,...,14.41,14.42,14.43,4200,16500,3000,7900,8400,14.46,7959
11858533,1607581806461705,104108485,22,600004,SH,143001910,5438227,78301011.00,14.39,14.37,...,14.41,14.42,14.43,4200,16500,3000,7900,8400,14.46,7959
11860322,1607581808756315,104124143,23,600004,SH,143004000,5438227,78301010.81,14.39,14.37,...,14.41,14.42,14.43,4200,19500,3000,7900,8400,14.46,7959
11861195,1607581809429503,104128800,22,600004,SH,143004910,5438227,78301011.00,14.39,14.38,...,14.41,14.42,14.43,4200,19500,3000,7900,8400,14.46,7959
11862939,1607581811766380,104144329,23,600004,SH,143007000,5438227,78301010.81,14.39,14.38,...,14.41,14.42,14.43,10900,19500,3000,7900,8400,14.46,7959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12910545,1607582997425513,112761620,22,600004,SH,144952980,5816998,83749127.00,14.38,14.38,...,14.41,14.42,14.43,6000,3500,14100,11300,8400,14.46,8701
12912127,1607582999440091,112775113,23,600004,SH,144954000,5816998,83749127.08,14.38,14.38,...,14.41,14.42,14.43,6000,3500,14100,11300,8400,14.46,8701
12914910,1607583002446813,112797210,23,600004,SH,144957000,5817198,83752003.08,14.38,14.38,...,14.41,14.42,14.43,6000,3500,14100,11300,8400,14.46,8702
12916129,1607583003443953,112805560,22,600004,SH,144959000,5817198,83752003.00,14.38,14.38,...,14.41,14.42,14.43,6000,3500,14100,11300,8400,14.46,8702


,clockAtArrival,sequenceNo,source,StockID,exchange,time,cum_volume,cum_amount,close,bid1p,...,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice,numTrades
9974100,1607581801627874,25174799,23,600004,SH,143001000,5438227,78301010.81,14.39,14.37,...,14.41,14.42,14.43,4200,16500,3000,7900,8400,14.46,7959
9974714,1607581802049105,25175965,22,600004,SH,143001910,5438227,78301011.00,14.39,14.37,...,14.41,14.42,14.43,4200,16500,3000,7900,8400,14.46,7959
9976219,1607581804347050,25179445,23,600004,SH,143004000,5438227,78301010.81,14.39,14.37,...,14.41,14.42,14.43,4200,19500,3000,7900,8400,14.46,7959
9976952,1607581805017498,25181133,22,600004,SH,143004910,5438227,78301011.00,14.39,14.38,...,14.41,14.42,14.43,4200,19500,3000,7900,8400,14.46,7959
9978433,1607581807361242,25184950,23,600004,SH,143007000,5438227,78301010.81,14.39,14.38,...,14.41,14.42,14.43,10900,19500,3000,7900,8400,14.46,7959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10867121,1607582993003329,27330482,22,600004,SH,144952980,5816998,83749127.00,14.38,14.38,...,14.41,14.42,14.43,6000,3500,14100,11300,8400,14.46,8701
10868456,1607582995021990,27333876,23,600004,SH,144954000,5816998,83749127.08,14.38,14.38,...,14.41,14.42,14.43,6000,3500,14100,11300,8400,14.46,8701
10870842,1607582998098990,27339594,23,600004,SH,144957000,5817198,83752003.08,14.38,14.38,...,14.41,14.42,14.43,6000,3500,14100,11300,8400,14.46,8702
10871882,1607582999021792,27341793,22,600004,SH,144959000,5817198,83752003.00,14.38,14.38,...,14.41,14.42,14.43,6000,3500,14100,11300,8400,14.46,8702


In [15]:
# in_dex = logSH[logSH["source"] == 5]["StockID"].unique()
in_dex = [16, 300, 852, 905]
data1 = logSH[~logSH["StockID"].isin(in_dex) & (logSH["cum_volume"] > 0) & (logSH['source'] == 23)]
data2 = logSH1[~logSH1["StockID"].isin(in_dex) & (logSH1['cum_volume'] > 0) & (logSH1['source'] == 23)]
columns = ["StockID", "cum_volume", "cum_amount", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q",
           "bid3q", "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
           "ask4q", "ask5q", 'numTrades', 'openPrice']
data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()
n1 = len(data1_1["StockID"].unique())
n2 = len(data2_1["StockID"].unique())

print(n1)
print(n2)
if n1 != n2:
    sl = list(set(data1_1["StockID"].unique()) & (set(data2_1["StockID"].unique())))
    data1_1 = data1_1[data1_1["StockID"].isin(sl)]
    data2_1 = data2_1[data2_1["StockID"].isin(sl)]
for cols in ['cum_amount']:
    data1_1[cols] = data1_1[cols].round(2)
    data2_1[cols] = data2_1[cols].round(2)
data1_1.loc[data1_1['cum_volume'] > 0, 'openPrice'] = data1_1[data1_1['cum_volume'] > 0].groupby('StockID')['openPrice'].transform('max')
data2_1.loc[data2_1['cum_volume'] > 0, 'openPrice'] = data2_1[data2_1['cum_volume'] > 0].groupby('StockID')['openPrice'].transform('max')
test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
n1 = test["sequenceNo_x"].count()
n2 = test["sequenceNo_y"].count()
len1 = len(test)
print(n1)
print(n2)
print(len1)
print("-----------------------------------------------")
if n2 < len1:
    display("test is not complete:")
    display(test[np.isnan(test["sequenceNo_y"])])
    print(len(test[np.isnan(test["sequenceNo_y"])])/n1)
if (len1 == n2) & (n1 < len1):
    display("baseline is not complete:")
    display(test[np.isnan(test["sequenceNo_x"])])
    display(n2-n1)
    print((n2-n1)/n1)

1755
1456
4881185
4881185
4881185
-----------------------------------------------


In [15]:
display(logSH[logSH['source'] == 25]['cum_amount'].astype('str').apply(lambda x: len(x.split('.')[1])).unique())
display(logSH1[logSH1['source'] == 22]['cum_amount'].round(2).astype('str').apply(lambda x: x.split('.')[1]).unique())

array([1, 2, 6, 7])

array(['0', '7', '5', '8', '2', '4', '1', '9', '6', '3'], dtype=object)

In [46]:
pd.set_option('max_columns', 200)
data1_1[(data1_1['StockID'] == 601018) & (data1_1['cum_volume'] >= 1491260)][['source', 'time'] + columns].head()

,source,time,StockID,cum_volume,cum_amount,close,bid1p,bid2p,bid3p,bid4p,bid5p,bid1q,bid2q,bid3q,bid4q,bid5q,ask1p,ask2p,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,numTrades,openPrice
213157,4,93652000,601018,1491260,5927719.6,3.96,3.96,3.95,3.94,3.93,3.92,274840,217100,197700,266700,185200,3.97,3.98,3.99,4.0,4.01,192900,182800,139200,171600,138700,398,3.98
214500,4,93655000,601018,1491360,5928116.6,3.97,3.96,3.95,3.94,3.93,3.92,274840,217100,202700,281800,185200,3.97,3.98,3.99,4.0,4.01,194900,182800,139200,171600,138700,399,3.98
215862,4,93658000,601018,1491360,5928116.6,3.97,3.96,3.95,3.94,3.93,3.92,275040,217100,202700,281800,185200,3.97,3.98,3.99,4.0,4.01,194900,182800,139200,171600,138700,399,3.98
217355,4,93701000,601018,1493960,5938412.6,3.96,3.96,3.95,3.94,3.93,3.92,272640,217100,202700,281800,185200,3.97,3.98,3.99,4.0,4.01,194900,184900,139200,171600,138700,402,3.98
219166,4,93704000,601018,1493960,5938412.6,3.96,3.96,3.95,3.94,3.93,3.92,273140,217600,202700,281800,185200,3.97,3.98,3.99,4.0,4.01,194900,184900,139600,171600,138700,402,3.98


In [48]:
data2_1[(data2_1['StockID'] == 600667) & (data2_1['cum_volume'] >= 5233518)][['source', 'time'] + columns]

,source,time,StockID,cum_volume,cum_amount,close,bid1p,bid2p,bid3p,bid4p,bid5p,bid1q,bid2q,bid3q,bid4q,bid5q,ask1p,ask2p,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,numTrades,openPrice
407897,23,94334000,600667,5233518,5.745340e+07,10.99,10.99,10.98,10.97,10.96,10.95,1400,26100,24800,49600,53000,11.00,11.01,11.02,11.03,11.04,54000,60300,116400,125100,81200,4132,10.93
409306,23,94337000,600667,5233518,5.745340e+07,10.99,10.99,10.98,10.97,10.96,10.95,7000,29800,24800,49600,52900,11.00,11.01,11.02,11.03,11.04,33800,60300,116400,125100,81200,4132,10.93
409309,23,94337000,600667,5234518,5.746439e+07,10.99,10.99,10.98,10.97,10.96,10.95,28700,29800,24800,49600,52900,11.00,11.01,11.02,11.03,11.04,33800,60300,116400,125100,81200,4135,10.93
410748,23,94340000,600667,5270518,5.786039e+07,11.00,11.00,10.99,10.98,10.97,10.96,5000,62300,29800,24800,49600,11.01,11.02,11.03,11.04,11.05,61100,115600,124600,81200,180700,4165,10.93
412134,23,94343000,600667,5347218,5.870487e+07,11.02,11.01,11.00,10.99,10.98,10.97,55300,10200,63300,29300,20800,11.02,11.03,11.04,11.05,11.06,113600,124600,81200,180700,109700,4205,10.93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2803051,23,111905000,600667,18416260,2.019556e+08,10.96,10.95,10.94,10.93,10.92,10.91,200,25500,106000,27313,57400,10.96,10.97,10.98,10.99,11.00,26000,21800,33500,50000,213900,16913,10.93
2804104,23,111908000,600667,18417060,2.019643e+08,10.96,10.95,10.94,10.93,10.92,10.91,2500,24200,106000,27313,57400,10.96,10.97,10.98,10.99,11.00,25200,21800,33500,50000,213900,16915,10.93
2805208,23,111911000,600667,18417060,2.019643e+08,10.96,10.95,10.94,10.93,10.92,10.91,4300,23900,106000,27313,57400,10.96,10.97,10.98,10.99,11.00,25200,21800,33500,50000,213900,16915,10.93
2806310,23,111914000,600667,18419160,2.019874e+08,10.96,10.95,10.94,10.93,10.92,10.91,4300,23900,105100,27313,57400,10.96,10.97,10.98,10.99,11.00,23600,21800,33500,50000,213900,16917,10.93


In [6]:
logSH1['time'].max()

112832710

In [21]:
data2['time'].min()

92500870

In [14]:
data1_1[(data1_1['cum_volume'] >= 35723) & (data1_1['StockID'] == 688177)].head()

,index,clockAtArrival,sequenceNo,source,StockID,exchange,time,cum_volume,cum_amount,close,...,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice,numTrades
67418,567135,1606440663312895,5884600,3,688177,SH,93212670,35723,1301656.0,36.28,...,36.38,36.40,36.43,461,2542,2742,800,1037,36.37,63
70272,573572,1606440669306249,6000553,3,688177,SH,93218660,45723,1663322.0,36.10,...,36.38,36.43,36.47,461,2078,2742,1037,2000,36.37,71
71706,576791,1606440672329392,6059267,3,688177,SH,93221680,45723,1663322.0,36.10,...,36.43,36.47,36.48,461,2742,1037,2000,2336,36.37,71
73149,580016,1606440675307792,6118179,3,688177,SH,93224670,52723,1916019.0,36.08,...,36.43,36.47,36.48,461,2742,1037,2000,2336,36.37,79
74587,583226,1606440678311051,6176810,3,688177,SH,93227670,52723,1916019.0,36.08,...,36.28,36.38,36.43,1153,3600,461,2742,1037,36.37,79


In [13]:
data2_1[(data2_1['cum_volume'] == 35723) & (data2_1['StockID'] == 688177)].head()

,index,clockAtArrival,sequenceNo,source,StockID,exchange,time,cum_volume,cum_amount,close,...,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice,numTrades
67418,568208,1606440663376130,1249773,22,688177,SH,93212670,35723,1301656.0,36.28,...,36.38,36.40,36.43,461,2542,2742,800,1037,36.37,63
68852,571444,1606440666382896,1261912,22,688177,SH,93215660,35723,1301656.0,36.28,...,36.38,36.43,36.47,461,2078,2742,1037,2000,36.37,63


In [5]:
# speed check
test1 = test[~np.isnan(test["sequenceNo_x"])]
test1 = test1[~np.isnan(test1["sequenceNo_y"])]
diff = test1["clockAtArrival_y"] - test1["clockAtArrival_x"]
print(np.quantile(abs(diff[diff < 0]), 0.25))
print(np.median(abs(diff[diff < 0])))
print(np.quantile(abs(diff[diff < 0]), 0.75))
print(np.quantile(diff[diff > 0], 0.25))
print(np.median(diff[diff > 0]))
print(np.quantile(diff[diff > 0], 0.75))
print(sum(diff < 0)/len(diff))
print(sum(diff > 0)/len(diff))

117.0
278.0
524.0
111.0
259.0
484.0
0.5040553123179956
0.49445108406647076


In [16]:
in_dex = [16, 300, 852, 905]
data1 = logSH[logSH["StockID"].isin(in_dex) & (logSH["cum_volume"] > 0)]
data2 = logSH1[logSH1["StockID"].isin(in_dex) & (logSH1["cum_volume"] > 0)]
# data2 = data2[data2["openPrice"] != 0]
columns = ["StockID", "cum_volume", "cum_amount", "close", "openPrice"]
data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()

for cols in ["cum_amount"]:
    data1_1[cols] = data1_1[cols].round(2)
    data2_1[cols] = data2_1[cols].round(2)

test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
n1 = test["sequenceNo_x"].count()
n2 = test["sequenceNo_y"].count()
len1 = len(test)
print(n1)
print(n2)
print(len1)
if n2 < len1:
    display("test is not complete:")
    display(test[np.isnan(test["sequenceNo_y"])])

11407
11407
11407


In [51]:
data1['StockID'].unique()

array([300, 852, 905])

In [19]:
test[test['sequenceNo_x'].isnull()]['time_y'].unique()

array([ 93007170,  93007180,  93012140,  93012150,  93017110,  93017120,
        93027170,  93027180,  93032140,  93032150,  93037110,  93037120,
        93047170,  93047180, 130007240, 130007250, 130012210, 130012220])

In [34]:
kk = test[(~test['sequenceNo_x'].isnull()) & (~test['sequenceNo_y'].isnull())][['time_x', 'time_y', 'clockAtArrival_x', 'clockAtArrival_y', "StockID", "cum_volume", "cum_amount"]]
kk['clockAtArrival_x'] - kk['clockAtArrival_y']
kk['time_x'] - kk['time_y']

0         -190.0
1         -190.0
2         -200.0
3         -170.0
4         -170.0
5         -180.0
6         -170.0
7         -170.0
8         -180.0
9         -140.0
10        -140.0
11        -150.0
12        -230.0
13        -240.0
14        -240.0
15        -220.0
16        -230.0
17        -230.0
18        -190.0
19        -200.0
20        -200.0
21        -160.0
22        -170.0
23        -170.0
24        -160.0
25        -170.0
26        -170.0
27        -250.0
28        -260.0
29        -260.0
30        -220.0
31        -230.0
32        -230.0
4329   -690540.0
4330   -695230.0
4331   -694930.0
dtype: float64

In [52]:
data2[data2['time'] > 94000000]['time'].min()

113701540

In [51]:
kk

,time_x,time_y,clockAtArrival_x,clockAtArrival_y,StockID,cum_volume,cum_amount
0,92521000.0,92521190.0,1.604971e+15,1.604971e+15,300,3183316,5.184215e+09
1,92521000.0,92521190.0,1.604971e+15,1.604971e+15,852,1762071,2.328115e+09
2,92521000.0,92521200.0,1.604971e+15,1.604971e+15,905,1685007,1.568999e+09
3,93001000.0,93001170.0,1.604972e+15,1.604972e+15,300,3323255,5.577770e+09
4,93001000.0,93001170.0,1.604972e+15,1.604972e+15,852,1801306,2.379085e+09
5,93001000.0,93001180.0,1.604972e+15,1.604972e+15,905,1721465,1.607525e+09
6,93006000.0,93006170.0,1.604972e+15,1.604972e+15,300,5305967,8.679816e+09
7,93006000.0,93006170.0,1.604972e+15,1.604972e+15,852,2859196,3.637863e+09
8,93006000.0,93006180.0,1.604972e+15,1.604972e+15,905,2857488,2.695295e+09
9,93011000.0,93011140.0,1.604972e+15,1.604972e+15,300,6679324,1.083663e+10


In [17]:
logSH1[(logSH1['StockID'] == 16) & (logSH1['cum_volume'] == 320135)]

,clockAtArrival,sequenceNo,source,StockID,exchange,time,cum_volume,cum_amount,close,bid1p,...,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice,numTrades
336725,1604453028526468,2323098,22,16,SH,92523100,320135,568550086.0,3311.5399,0.0,...,0.0,0.0,0.0,0,0,0,0,0,3311.5399,0
338046,1604453045012103,2325765,22,16,SH,92539500,320135,568550086.0,3311.5399,0.0,...,0.0,0.0,0.0,0,0,0,0,0,3311.5399,0
340516,1604453073588981,2335156,22,16,SH,92608160,320135,568550086.0,3311.5399,0.0,...,0.0,0.0,0.0,0,0,0,0,0,3311.5399,0
340599,1604453075021480,2335348,22,16,SH,92609490,320135,568550086.0,3311.5399,0.0,...,0.0,0.0,0.0,0,0,0,0,0,3311.5399,0
343053,1604453105027232,2340255,22,16,SH,92639590,320135,568550086.0,3311.5399,0.0,...,0.0,0.0,0.0,0,0,0,0,0,3311.5399,0
345523,1604453133563997,2349646,22,16,SH,92708130,320135,568550086.0,3311.5399,0.0,...,0.0,0.0,0.0,0,0,0,0,0,3311.5399,0
345606,1604453135037387,2349837,22,16,SH,92709500,320135,568550086.0,3311.5399,0.0,...,0.0,0.0,0.0,0,0,0,0,0,3311.5399,0
348060,1604453165048274,2354745,22,16,SH,92739500,320135,568550086.0,3311.5399,0.0,...,0.0,0.0,0.0,0,0,0,0,0,3311.5399,0
350530,1604453193536216,2364136,22,16,SH,92808110,320135,568550086.0,3311.5399,0.0,...,0.0,0.0,0.0,0,0,0,0,0,3311.5399,0
350613,1604453195076125,2364330,22,16,SH,92809500,320135,568550086.0,3311.5399,0.0,...,0.0,0.0,0.0,0,0,0,0,0,3311.5399,0


In [7]:
# speed check
test1 = test[~np.isnan(test["sequenceNo_x"])]
test1 = test1[~np.isnan(test1["sequenceNo_y"])]
diff = test1["clockAtArrival_y"] - test1["clockAtArrival_x"]
print(np.quantile(abs(diff[diff < 0]), 0.25))
print(np.median(abs(diff[diff < 0])))
print(np.quantile(abs(diff[diff < 0]), 0.75))
print(np.quantile(diff[diff > 0], 0.25))
print(np.median(diff[diff > 0]))
print(np.quantile(diff[diff > 0], 0.75))
print(sum(diff < 0)/len(diff))
print(sum(diff > 0)/len(diff))

122.0
280.5
513.0
114.0
260.0
490.0
0.501023040654746
0.4981763188328441


In [37]:
test1[test1['clockAtArrival_y'] - test1['clockAtArrival_x'] < 0].groupby('StockID')['time_x'].describe().astype('int64')

,count,mean,std,min,25%,50%,75%,max
StockID,,,,,,,,
16,2297,120059117,18170601,92522000,103547000,112617000,135417000,150027000
300,2116,120379811,17929594,92522000,104044500,112849500,135268250,150027000
852,2077,120315586,17901461,92522000,104017000,112832000,135222000,150027000
905,2077,120315586,17901461,92522000,104017000,112832000,135222000,150027000


In [38]:
pd.set_option('max_columns', 200)
logSH[(logSH['StockID'] == 852) & (logSH['cum_volume'] >= 12783347)].head(100)

,clockAtArrival,sequenceNo,source,StockID,exchange,time,cum_volume,cum_amount,close,bid1p,bid2p,bid3p,bid4p,bid5p,bid1q,bid2q,bid3q,bid4q,bid5q,ask1p,ask2p,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice,numTrades
272177,1606959357190775,9865837,26,852,SH,93557000,12783347,1.278460e+10,6714.5915,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,6726.2601,0
274373,1606959362165553,9980507,26,852,SH,93602000,12911763,1.291738e+10,6713.9975,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,6726.2601,0
276524,1606959367289176,10101091,26,852,SH,93607000,13053356,1.307422e+10,6714.2099,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,6726.2601,0
278988,1606959372225849,10209436,26,852,SH,93612000,13254160,1.327523e+10,6713.2415,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,6726.2601,0
281234,1606959377195646,10320007,26,852,SH,93617000,13380070,1.340864e+10,6713.6419,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,6726.2601,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
467791,1606959797249871,17527369,26,852,SH,94317000,22927889,2.377233e+10,6695.2535,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,6726.2601,0
469789,1606959802277744,17602308,26,852,SH,94322000,23010289,2.386869e+10,6696.3702,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,6726.2601,0
472134,1606959807234462,17670929,26,852,SH,94327000,23124755,2.399078e+10,6696.9714,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,6726.2601,0
474289,1606959812187964,17740355,26,852,SH,94332000,23206939,2.407627e+10,6697.5045,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,6726.2601,0


In [45]:
pd.set_option('max_columns', 200)
logSH1[(logSH1['StockID'] == 852) & (logSH1['cum_volume'] >= 12783347)].head(20)

,clockAtArrival,sequenceNo,source,StockID,exchange,time,cum_volume,cum_amount,close,bid1p,bid2p,bid3p,bid4p,bid5p,bid1q,bid2q,bid3q,bid4q,bid5q,ask1p,ask2p,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice,numTrades
580418,1606959357272169,2046080,22,852,SH,93557160,12783347,1.278460e+10,6714.5915,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,6726.2601,0
584514,1606959362154795,2068706,22,852,SH,93602130,12911763,1.291738e+10,6713.9975,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,6726.2601,0
589278,1606959367350318,2096257,22,852,SH,93607220,13053356,1.307422e+10,6714.2099,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,6726.2601,0
594311,1606959372242517,2122898,22,852,SH,93612190,13254160,1.327523e+10,6713.2415,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,6726.2601,0
597848,1606959376699265,2141966,22,852,SH,93616670,13254160,1.327523e+10,6713.2415,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,6726.2601,0
598455,1606959377182689,2144613,22,852,SH,93617160,13380070,1.340864e+10,6713.6419,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,6726.2601,0
602835,1606959382176413,2167437,22,852,SH,93622130,13508415,1.355831e+10,6714.7232,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,6726.2601,0
608000,1606959387514956,2191454,22,852,SH,93627220,13705314,1.376530e+10,6717.1339,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,6726.2601,0
612214,1606959392239605,2213393,22,852,SH,93632220,13860693,1.390913e+10,6718.9335,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,6726.2601,0
616805,1606959397260920,2235341,22,852,SH,93637190,14006306,1.406511e+10,6720.5803,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,6726.2601,0


In [43]:
test[(test['StockID'] == 852) & (test['clockAtArrival_y'] - test['clockAtArrival_x'] < 0)].head(60)

,index_x,clockAtArrival_x,sequenceNo_x,source_x,StockID,exchange_x,time_x,cum_volume,cum_amount,close,bid1p_x,bid2p_x,bid3p_x,bid4p_x,bid5p_x,bid1q_x,bid2q_x,bid3q_x,bid4q_x,bid5q_x,ask1p_x,ask2p_x,ask3p_x,ask4p_x,ask5p_x,ask1q_x,ask2q_x,ask3q_x,ask4q_x,ask5q_x,openPrice,numTrades_x,index_y,clockAtArrival_y,sequenceNo_y,source_y,exchange_y,time_y,bid1p_y,bid2p_y,bid3p_y,bid4p_y,bid5p_y,bid1q_y,bid2q_y,bid3q_y,bid4q_y,bid5q_y,ask1p_y,ask2p_y,ask3p_y,ask4p_y,ask5p_y,ask1q_y,ask2q_y,ask3q_y,ask4q_y,ask5q_y,numTrades_y
2,101250,1606958722180723,1616364,26,852,SH,92522000,1269012,1.153399e+09,6726.2601,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,6726.2601,0,228189,1606958722180643,327262,22,SH,92522150,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0
6,109548,1606959002209443,1919592,26,852,SH,93002000,1420860,1.256451e+09,6724.9741,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,6726.2601,0,250673,1606959002200743,374612,22,SH,93002180,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0
10,112477,1606959007229436,2586499,26,852,SH,93007000,2321415,2.084428e+09,6722.8635,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,6726.2601,0,256255,1606959007229385,423169,22,SH,93007160,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0
14,114668,1606959012453299,2854678,26,852,SH,93012000,2861941,2.629415e+09,6718.1545,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,6726.2601,0,261573,1606959012449043,472606,22,SH,93012250,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0
18,116812,1606959017239440,3008371,26,852,SH,93017000,3053852,2.820839e+09,6715.6007,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,6726.2601,0,265733,1606959017230012,520725,22,SH,93017210,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0
26,121704,1606959027181728,3280107,26,852,SH,93027000,3429223,3.193572e+09,6712.2601,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,6726.2601,0,275877,1606959027181662,610733,22,SH,93027160,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0
30,123834,1606959032310013,3412969,26,852,SH,93032000,3593238,3.342918e+09,6711.3173,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,6726.2601,0,280183,1606959032291137,636437,22,SH,93032250,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0
34,126352,1606959037244232,3539422,26,852,SH,93037000,3770471,3.527179e+09,6712.7749,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,6726.2601,0,285185,1606959037241339,665457,22,SH,93037220,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0
38,128788,1606959042220053,3653355,26,852,SH,93042000,4002784,3.770304e+09,6715.0811,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,6726.2601,0,290259,1606959042214371,688789,22,SH,93042190,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0
42,130823,1606959047184801,3772186,26,852,SH,93047000,4175301,3.935050e+09,6716.1012,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,6726.2601,0,294297,1606959047184697,711766,22,SH,93047160,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0


In [1]:
import pandas as pd
SH = pd.read_csv('/mnt/data/20201210/logs_20201210_zt_88_03_day_88data/mdTradeLog_20201210_0832.csv',
                    encoding="utf-8", index_col=False)
SH1 = pd.read_csv('/mnt/data/20201210/zs_7201/zs_7201_20201210_10.0.98.140_postvde/Logs_full/mdTradeLog_20201210_0854.csv',
                    encoding="utf-8", index_col=False)
SH = SH[SH["exchId"] == 1]
SH1 = SH1[SH1["exchId"] == 1]

In [2]:
import numpy as np
SH["ExecType"] = SH["ExecType"].apply(lambda x: str(x))
SH1["ExecType"] = SH1["ExecType"].apply(lambda x: str(x))
# ####
# SH = SH[SH['TransactTime'] <= SH1['TransactTime'].max()]
# ####

columns = ["TransactTime", "ApplSeqNum", "SecurityID", "TradePrice", "TradeQty", "TradeMoney", "TradeBSFlag","ExecType",
           "BidApplSeqNum", "OfferApplSeqNum", 'secid', 'exchId']
# ####
# SH1 = SH1.drop_duplicates(columns, keep='first')
# ####


n1 = len(SH["SecurityID"].unique())
n2 = len(SH1["SecurityID"].unique())
print(n1)
print(n2)
print(set(SH["SecurityID"].unique()) - set(SH1["SecurityID"].unique()))
print(set(SH1["SecurityID"].unique()) - set(SH["SecurityID"].unique()))
if n1 != n2:
    sl = list(set(SH["SecurityID"].unique()) & set(SH1["SecurityID"].unique()))
    SH = SH[SH["SecurityID"].isin(sl)]
    SH1 = SH1[SH1["SecurityID"].isin(sl)]
re = pd.merge(SH, SH1, left_on=columns, right_on=columns, how="outer", validate='one_to_one')
n1 = re["sequenceNo_x"].count()
n2 = re["sequenceNo_y"].count()
len1 = len(re)
print(n1)
print(n2)
print(len1)
print("-----------------------------------------------")
if n2 < len1:
    display("test is not complete:")
    display(re[np.isnan(re["sequenceNo_y"])])
    print(len(re[np.isnan(re["sequenceNo_y"])]))
    print(np.sort(re[np.isnan(re["sequenceNo_y"])]["TransactTime"].unique()))
    print(len(re[np.isnan(re["sequenceNo_y"])]["SecurityID"].unique()))
    print(re[np.isnan(re["sequenceNo_y"])]["SecurityID"].unique())
if (len1 == n2) & (n1 < len1):
    display("baseline is not complete:")
    display(re[np.isnan(re["sequenceNo_x"])])
    print(np.sort(re[np.isnan(re["sequenceNo_x"])]["TransactTime"].unique()))
    print(len(re[np.isnan(re["sequenceNo_x"])]["SecurityID"].unique()))
    print(re[np.isnan(re["sequenceNo_x"])]["SecurityID"].unique())
    display(n2-n1)

1755
1456
{688128, 688129, 605188, 688133, 688135, 601609, 688138, 688139, 688028, 688122, 603662, 605198, 605199, 688029, 603155, 688030, 688155, 688156, 688157, 688158, 688159, 688160, 603682, 605218, 688165, 688166, 603687, 688168, 688169, 605222, 688177, 688178, 688179, 688180, 688181, 688037, 688185, 688186, 601658, 688188, 688189, 603195, 688039, 688196, 688198, 688199, 688200, 603719, 688202, 605255, 603212, 605258, 688208, 603221, 688215, 688218, 688219, 688221, 688222, 601696, 601187, 688228, 688229, 601702, 603236, 605288, 688233, 603755, 603256, 688256, 688258, 605318, 688127, 688266, 603786, 688268, 603279, 688788, 688277, 688278, 601236, 605333, 605336, 603290, 605338, 688286, 688288, 688289, 688058, 603815, 688298, 688299, 688300, 688301, 605358, 688060, 601778, 688308, 688309, 688310, 688311, 688312, 688313, 605366, 605369, 688318, 605376, 688321, 688330, 605388, 688333, 688335, 688336, 688338, 688339, 605399, 601816, 603353, 601827, 688356, 688357, 688358, 688360, 68836

In [11]:
# speed check
re1 = re[~np.isnan(re["sequenceNo_x"])]
re1 = re1[~np.isnan(re1["sequenceNo_y"])]
diff = re1["clockAtArrival_y"] - re1["clockAtArrival_x"]
print(np.quantile(abs(diff[diff < 0]), 0.25))
print(np.median(abs(diff[diff < 0])))
print(np.quantile(abs(diff[diff < 0]), 0.75))
print(np.quantile(diff[diff > 0], 0.25))
print(np.median(diff[diff > 0]))
print(np.quantile(diff[diff > 0], 0.75))
print(sum(diff < 0)/len(diff))
print(sum(diff > 0)/len(diff))

131.0
296.0
532.0
126.0
275.0
492.0
0.4970849055007425
0.5018735085020862


In [11]:
re1[(re1['clockAtArrival_y'] - re1['clockAtArrival_x'] >= 3086)]

,clockAtArrival_x,sequenceNo_x,exchId,securityType_x,__isRepeated_x,TransactTime,ChannelNo_x,ApplSeqNum,SecurityID,secid,...,TradeMoney,BidApplSeqNum,OfferApplSeqNum,clockAtArrival_y,sequenceNo_y,securityType_y,__isRepeated_y,ChannelNo_y,mdSource_y,__origTickSeq_y
30875,1606872238141382,2059980,1,1,0,92500170,4,10955,600004,1600004,...,76456000,95773,95500,1.606872e+15,442336.0,1.0,0.0,4.0,23.0,-1.0
30876,1606872238141383,2059981,1,1,0,92500170,4,10956,600004,1600004,...,75544000,95773,97859,1.606872e+15,442337.0,1.0,0.0,4.0,23.0,-1.0
30877,1606872238141386,2059982,1,1,0,92500170,4,10957,600004,1600004,...,15200000,102888,97859,1.606872e+15,442338.0,1.0,0.0,4.0,23.0,-1.0
30878,1606872238141386,2059983,1,1,0,92500170,4,10958,600004,1600004,...,61256000,136868,97859,1.606872e+15,442339.0,1.0,0.0,4.0,23.0,-1.0
30879,1606872238141387,2059984,1,1,0,92500170,4,10959,600004,1600004,...,60800000,136868,101494,1.606872e+15,442340.0,1.0,0.0,4.0,23.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13410440,1606879649005743,84939637,1,1,0,112832670,6,2687617,600060,1600060,...,94240000,4538880,4532104,1.606881e+15,40772873.0,1.0,0.0,6.0,23.0,-1.0
13410441,1606879649005745,84939638,1,1,0,112832670,6,2687618,600060,1600060,...,117800000,4538880,4532117,1.606881e+15,40772874.0,1.0,0.0,6.0,23.0,-1.0
13410442,1606879649005747,84939639,1,1,0,112832670,6,2687619,600060,1600060,...,11780000,4538880,4533222,1.606881e+15,40772875.0,1.0,0.0,6.0,23.0,-1.0
13410443,1606879649005748,84939640,1,1,0,112832670,6,2687620,601808,1601808,...,13210000,4247824,4538882,1.606881e+15,40772876.0,1.0,0.0,6.0,23.0,-1.0


In [13]:
SH[(SH['SecurityID'] == 600004) & (SH['ApplSeqNum'] == 10955)]

,clockAtArrival,sequenceNo,exchId,securityType,__isRepeated,TransactTime,ChannelNo,ApplSeqNum,SecurityID,secid,mdSource,ExecType,TradeBSFlag,__origTickSeq,TradePrice,TradeQty,TradeMoney,BidApplSeqNum,OfferApplSeqNum
89333,1606872238141382,2059980,1,1,0,92500170,4,10955,600004,1600004,4,F,N,7686860,152000,503,76456000,95773,95500


In [14]:
SH1[(SH1['SecurityID'] == 600004) & (SH1['ApplSeqNum'] == 10955)]

,clockAtArrival,sequenceNo,exchId,securityType,__isRepeated,TransactTime,ChannelNo,ApplSeqNum,SecurityID,secid,mdSource,ExecType,TradeBSFlag,__origTickSeq,TradePrice,TradeQty,TradeMoney,BidApplSeqNum,OfferApplSeqNum
30875,1606872238144469,442336,1,1,0,92500170,4,10955,600004,1600004,23,F,N,-1,152000,503,76456000,95773,95500


In [19]:
stocks = SH["SecurityID"].unique()
m = 0
diff = []
for i in stocks:
    re1 = pd.merge(SH[SH["SecurityID"] == i], SH1[SH1["SecurityID"] == i], left_on=columns, right_on=columns, how="inner")
    if (sorted(re1["sequenceNo_y"]) != re1["sequenceNo_y"]).any() == False:
        m = m + 1
    else:
        diff.append(i)
if m == len(stocks):
    print("SHE Trade Data have same ordering within stocks")
else:
    print("SHE Trade Data don't have same ordering within stocks")

SHE Trade Data have same ordering within stocks
